# This document helps understand future RA's how the dynamic programming algorithm works by using random data and running it for only a single time period.

In [1]:
μ_min1 = Array{Float64}(undef, 2, 2, 2, 2)
π0 = Array{Float64}(undef, 2 ,2 , 2, 2)
β = 0.8
ν = 1.1

1.1

In [2]:
using Random
Random.seed!(1)

TaskLocalRNG()

In [3]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        μ_min1[i, j, :, :] = matrix2
    end
end
μ_min1

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0419308  0.344161
 0.359995   0.203362

[:, :, 2, 1] =
 0.199601   0.0725269
 0.0900771  0.000908277

[:, :, 1, 2] =
 0.399398  0.289712
 0.306728  0.381414

[:, :, 2, 2] =
 0.35907   0.293601
 0.243201  0.414316

In [4]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        π0[i, j, :, :] = matrix2
    end
end
π0

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.308254  0.238938
 0.187336  0.354732

[:, :, 2, 1] =
 0.0196254  0.272704
 0.341713   0.298983

[:, :, 1, 2] =
 0.491333  0.118526
 0.141249  0.17604

[:, :, 2, 2] =
 0.180788  0.369832
 0.329702  0.170244

In [5]:
κdot = rand(0.95:0.001:1.08, (2, 2, 2, 2))

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 1.055  0.995
 1.036  1.051

[:, :, 2, 1] =
 0.975  0.951
 1.007  1.08

[:, :, 1, 2] =
 1.035  1.058
 1.006  1.001

[:, :, 2, 2] =
 1.007  0.958
 0.964  1.076

In [6]:
Ȧ = rand(0.98:0.001:1.05, (2, 2))

2×2 Matrix{Float64}:
 0.999  1.028
 1.024  0.986

In [7]:
L_setup_matrix = rand(2, 2)
L0 = L_setup_matrix/sum(L_setup_matrix)

2×2 Matrix{Float64}:
 0.274791  0.269369
 0.069633  0.386207

In [8]:
w0 = rand(0.5:0.001:1.5, (2, 2))

2×2 Matrix{Float64}:
 1.165  1.406
 1.183  1.303

In [9]:
using Distributions
θ_var = Frechet()
θ_list = rand(θ_var, 2)

2-element Vector{Float64}:
 0.8961287769725773
 0.7719921849322902

In [10]:
u̇_0t = rand(0.96:0.001:1.04, (2, 2, 2))
# here, u̇_0t is the the initial guess for u̇ in both the time periods, where u̇_0t[:, :, 1] is for the first time period
# and u̇_0t[:, :, 2] is for the second time period.

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.969  1.008
 0.992  1.023

[:, :, 2] =
 0.976  0.983
 0.98   1.034

In [11]:
ẇ = rand(0.96:0.001:1.04, (2, 2))

2×2 Matrix{Float64}:
 1.04   0.963
 1.024  0.992

In [12]:
μ_0 = Array{Float64}(undef, 2, 2, 2, 2)
L1 = Array{Float64}(undef, 2, 2)
ẇn = Array{Float64}(undef, 2, 2)#This is going to be our updated wage change guess.
Ṗ = Array{Float64}(undef, 2, 2)
π1 = Array{Float64}(undef, 2, 2, 2, 2)
X = Array{Float64}(undef, 2, 2)
α = Array{Float64}(undef, 2)
Xt1 = Array{Float64}(undef, 2, 2)
ẇ_old = Array{Float64}(undef, 2, 2) # This variable will be used to store the initial value of ẇ at the beginning of each iteration
u̇_0t_old = Array{Float64}(undef, 2, 2) #This variable will be used to store the initial value of u̇_0t at the beginning of each iteration.
u̇_1t = Array{Float64}(undef, 2, 2)#here, u̇_1t is our updated guess for u̇_0t[:, :, 1], that is u̇_0t in the first period.

2×2 Matrix{Float64}:
 9.69967e-312  NaN
 3.838e-320      0.0

In [13]:
while !all(0.99 .*u̇_0t[:, :, 1] .<= u̇_0t_old .<= u̇_0t[:, :, 1] )
    println("NEXT ITERATION STARTS")
    println("u̇_0t at the beginning of the new iteration")
    println(u̇_0t)
    println("old u̇_0t")
    println(u̇_0t_old)
    u̇_0t_old .= deepcopy(u̇_0t[:, :, 1]) # updating u̇_0t_old
    println("u̇_0t_old updated")
    println(u̇_0t_old)
    for n in 1:2
        for j in 1:2
            for i in 1:2
                for k in 1:2
                    denom = sum(μ_min1[n, j, m, h]*((u̇_0t[m, h, 1])^(β/ν)) for m in 1:2, h in 1:2)# has to be 2
                    print("denom ", denom)
                    numer = (μ_min1[n, j, i, k]*((u̇_0t[i, k, 1])^(β/ν))) # has to be 2
                    print("numer ", numer)
                    μ_0[n, j, i, k] = numer/denom
                end
            end
        end
    end
    println("μ_0")
    println(μ_0)
    for n in 1:2
        for j in 1:2
            L1[n, j] = sum(μ_0[i, k, n, j]*L0[i, k] for i in 1:2, k in 1:2)
        end
    end
    println("L1")
    println(L1)
    println("ẇ_old before the loop starts")
    println(ẇ_old)
    println("ẇ before the loop starts")
    println(ẇ)
    while !all(0.99*ẇ .<= ẇ_old .<= 1.01*ẇ)
        println(":::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")
        println("INNER LOOP STARTS")
        ẇ_old .= deepcopy(ẇ)
        println("ẇ_old updated")
        println(ẇ_old)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    Ṗ[n, j] = (sum(π0[n, j, i, j]*((ẇ[i, j]*κdot[n, j, i, j])^(-θ_list[j]))*((Ȧ[i, j])^θ_list[j]) for i in 1:2))^(-1/θ_list[j])   
                end
            end
        end
        println("Ṗ")
        println(Ṗ)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    π1[n, j, i, j] = π0[n, j, i, j]*(((ẇ[i, j]*κdot[n, j, i, j])/Ṗ[n, j])^(-θ_list[j]))*((Ȧ[i,j])^θ_list[j])
                end
            end
        end
        println("π1")
        println(π1)
        for i in 1:2
            for j in 1:2
                X[i, j] = w0[i, j]*L0[i, j]
            end
        end
        println("X")
        println(X)
        for i in 1:2
            α[i] = sum(X[k, i] for k in 1:2)/sum(X[j, l] for j in 1:2, l in 1:2)
        end
        println("α")
        println(α)
        for n in 1:2
            for j in 1:2
                Xt1[n, j] = α[j]*sum(ẇ[n, k]*L1[n, k]*w0[n, k] for k in 1:2)
            end
        end
        for n in 1:2
            for j in 1:2
                ẇn[n, j] = sum(π1[i, j, n, j]*Xt1[i, j] for i in 1:2)/(L1[n, j]*w0[n, j])
            end
        end
        println("ẇn at the end of inner loop")
        println(ẇn)
        ẇ .= ẇn
        println("ẇ reassigned")
        println(ẇ)
        println("INNER LOOP ENDS")
        println(":::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")
    end
    println("ẇn after the loop ends")
    println(ẇn)
    println("ẇ after the loop ends")
    println(ẇ)
    for n in 1:2
        for j in 1:2
            u̇_1t[n, j] = ẇ[n, j]*(sum(μ_0[n, j, i, k]*(u̇_0t[i, k, 2])^(β/ν) for i in 1:2, k in 1:2))^(ν)
        end
    end
    println("u̇_0t before the loop")
    println(u̇_0t)
    println("u̇_1t")
    println(u̇_1t)
    # HERE FIRST STORE VALUE OF INITIAL UDOT FROM THE SECOND PERIOD INTO 
    u̇_0t[:, :, 1] .= u̇_1t
    println("u̇_0t updated and here it is after the loop")
    println(u̇_0t)
    println("ITERATION ENDS")
    println("------------------------------------------------------------------------------------------------")
    println("------------------------------------------------------------------------------------------------")
    println("------------------------------------------------------------------------------------------------")
end 

NEXT ITERATION STARTS
u̇_0t at the beginning of the new iteration
[0.969 1.008; 0.992 1.023;;; 0.976 0.983; 0.98 1.034]
old u̇_0t
[5.0e-324 1.5e-323; 1.0e-323 2.0e-323]
u̇_0t_old updated
[0.969 1.008; 0.992 1.023]
denom 1.0061968674175936numer 0.04098143727810674denom 1.0061968674175936numer 0.40171941355015744denom 1.0061968674175936numer 0.19843828089754162denom 1.0061968674175936numer 0.36505773569178773denom 0.9983647178427981numer 0.33636803817133654denom 0.9983647178427981numer 0.2913955092896981denom 0.9983647178427981numer 0.07210444143891757denom 0.9983647178427981numer 0.298496728942846denom 0.9971624342718468numer 0.3518436185678454denom 0.9971624342718468numer 0.3085101962725042denom 0.9971624342718468numer 0.08955247715309733denom 0.9971624342718468numer 0.24725614227839982denom 1.0045157549787054numer 0.198757422784896denom 1.0045157549787054numer 0.38363030135993037denom 1.0045157549787054numer 0.0009029863038199812denom 1.0045157549787054numer 0.42122504453005916μ_0
[0.

[0.32013176774301466 0.37873319613287343; 0.08237583893562075 0.5032270842504509]
α
[0.31336525477434485, 0.6866347452256553]
ẇn at the end of inner loop
[1.165816267340471 0.7477590975792043; 1.3600407561624575 1.0912889075373646]
ẇ reassigned
[1.165816267340471 0.7477590975792043; 1.3600407561624575 1.0912889075373646]
INNER LOOP ENDS
:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
INNER LOOP STARTS
ẇ_old updated
[1.165816267340471 0.7477590975792043; 1.3600407561624575 1.0912889075373646]
Ṗ
[4.285715588250443 2.4634693012003486; 2.623298147415124 3.579195183897657]
π1
[0.9426418004699685 6.95251268592183e-310; 0.37505949928881943 6.95251268592183e-310;;; 0.057358199530031684 6.95251049453357e-310; 0.6249405007111807 0.0;;;; 6.95251049595016e-310 0.2910020084775406; 6.95251049453357e-310 0.6018784207662836;;; 6.95251049453357e-310 0.7089979915224596; 0

u̇_0t before the loop
[0.969 1.008; 0.992 1.023;;; 0.976 0.983; 0.98 1.034]
u̇_1t
[1.145799513174277 0.7681199382737149; 1.4030217351779435 1.0690013532317946]
u̇_0t updated and here it is after the loop
[1.145799513174277 0.7681199382737149; 1.4030217351779435 1.0690013532317946;;; 0.976 0.983; 0.98 1.034]
ITERATION ENDS
------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------
NEXT ITERATION STARTS
u̇_0t at the beginning of the new iteration
[1.145799513174277 0.7681199382737149; 1.4030217351779435 1.0690013532317946;;; 0.976 0.983; 0.98 1.034]
old u̇_0t
[0.969 1.008; 0.992 1.023]
u̇_0t_old updated
[1.145799513174277 0.7681199382737149; 1.4030217351779435 1.0690013532317946]
denom 1.0082296276638023numer 0.046293671614707994denom 1.0082296276638023numer 0